In [1]:
import pandas as pd
from RoutePlanner.Optimisation import TravelTime
from RoutePlanner import Plot
from RoutePlanner.TemporalCellGrid import TemporalCellGrid
import pickle

In [4]:
OptInfo = {}

# Mesh Information
OptInfo['Mesh']  = {}
OptInfo['Mesh']['Longitude Bounds (Min,Max,Width)']      = [-130,25,5]
OptInfo['Mesh']['Latitude Bounds (Min,Max,Width)']       = [-80,-40,2.5]

OptInfo['Mesh']['Date Range (Min,Max,dT)']               = ['2016-01-1','2016-02-1',1.0]
OptInfo['Mesh']['Homogenous Params (Threshold,Min,Max)'] = [0.12,0.05,0.85] #[0.04,0.05,0.85] - Slow Vehicle, [0.12,0.05,0.85] - SDA
OptInfo['Mesh']['Current Data Path']                     = "../Data/SOSE_surface_velocity_6yearMean_2005-2010.nc"
OptInfo['Mesh']['Ice Data Path']                         = "../Data/bsose_i122_2013to2017_1day_SeaIceArea.nc"

# Route Information
OptInfo['Route'] = {} 
OptInfo['Route']['WayPoints']            = '../resources/WayPoints_org.csv'
OptInfo['Route']['MaxIceExtent']         = 0.8
OptInfo['Route']['Zero Currents']        = True
OptInfo['Route']['VariableSpeed']        = False
OptInfo['Route']['Time Unit']            = 'days'

OptInfo['Route']['VehicleInfo']                = {}
OptInfo['Route']['VehicleInfo']['Speed']       = 26.5
OptInfo['Route']['VehicleInfo']['Unit']        = 'km/hr'
OptInfo['Route']['VehicleInfo']['Beam']        = 24.0
OptInfo['Route']['VehicleInfo']['HullType']    = 'slender'
OptInfo['Route']['VehicleInfo']['ForceLimit']  = 96634.5

In [11]:
# Generating Cell Grid
temporalCellGrid = TemporalCellGrid(OptInfo)
cellGrid         = temporalCellGrid.range(OptInfo['Mesh']['Date Range (Min,Max,dT)'][0],OptInfo['Mesh']['Date Range (Min,Max,dT)'][1],j_grid=True)
cellGrid.iterativeSplit(0)

In [18]:
LegacyWaypoints = pd.read_csv('WayPointsLog.csv')
LegacyWaypoints


,Name,Lat,Long,Long(o),Lat(o)
0,A,-65.00069,-26.10194,-26°6'7'',-65°00'25''
1,B,-65.30139,-28.00000,-28°00'0'',-65°18'5''
2,C,-65.10000,-28.60000,-28°36'0'',-65°06'0''
3,D,-64.39583,-28.90028,-28°54'1'',-64°23'45''


In [30]:
def MapLegacyPath(points,map,color='blue',ShowPath=True,PathPoints=True):
    import folium
    import copy    
    Pths        = folium.FeatureGroup(name='Path')
    Pths_points = folium.FeatureGroup(name='Path Points')
    points = points[:,::-1]
    folium.PolyLine(points,color=color, weight=2.0, opacity=1).add_to(Pths)
    for idx in range(len(points)):
        loc = [points[idx,0],points[idx,1]]
        folium.Marker(
            location=loc,
            icon=folium.plugins.BeautifyIcon(icon='circle',
                                        border_color='transparent',
                                        background_color='transparent',
                                        border_width=1,
                                        text_color=color,
                                        inner_icon_style='margin:0px;font-size:0.8em')
        ).add_to(Pths_points)
    if ShowPath:
        Pths.add_to(map)
    if PathPoints:
        Pths_points.add_to(map)
    return map

file ='FrSt_geo.json'

def JSON2Numpy(file):
    # Reading Legacy Paths
    import json
    import numpy as np

    with open(file, 'r') as f:
            JavaGeo = json.load(f)
    return np.array(JavaGeo['features'][0]['geometry']['coordinates'])



map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map)
map = MapLegacyPath(LegacyWaypoints[['Long','Lat']].to_numpy(),map,ShowPath=True,PathPoints=True)

map = MapLegacyPath(JSON2Numpy(file),map,color='black',ShowPath=True,PathPoints=False)

map = Plot.LayerControl(map,collapsed=True)
map

/Users/jsmith/miniconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/jsmith/Documents/Research/Researcher_BAS/RoutePlanning/SDADT-pyRoutePlanner/RoutePlanner/IO.py:180: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Polygons['Land'][np.isnan(Polygons['Ice Area'])] = True


In [ ]:
def JavaPaths(file):
    import json
    import numpy as np
    with open(file, 'r') as f:
        JavaGeo = json.load(f)['allpaths']['January']

    PathPts = []
    for path in JavaGeo:
        if any(path['from'] in s for s in TT.source_waypoints) and any(path['to'] in s for s in TT.end_waypoints):
            print(path['from'],path['to'])
            pts=[]
            for jj in path['path']:
                try:
                    pts.append([jj['lon'],-jj['lat']])
                except:
                    continue
            pts = np.array(pts)
            PathPts.append(pts)
    return PathPts


array([[-25.3339, -65.2838],
       [-25.6856, -65.1352],
       [-26.1019, -65.0005],
       [-26.5344, -65.0669],
       [-27.0005, -65.1341],
       [-27.4858, -65.2181],
       [-27.9683, -65.2856],
       [-28.35  , -65.1839],
       [-28.6169, -65.0181],
       [-28.7006, -64.85  ],
       [-28.7681, -64.6675],
       [-28.8506, -64.4842],
       [-29.0683, -64.3183],
       [-29.4178, -64.1853],
       [-29.7839, -64.0667],
       [-30.1189, -63.9339],
       [-30.4513, -63.8019],
       [-30.8339, -63.6678],
       [-31.1681, -63.5353],
       [-31.5022, -63.4022],
       [-31.8678, -63.2689],
       [-32.2   , -63.1508],
       [-32.535 , -63.0178],
       [-32.9353, -62.8842]])

In [ ]:
map = Plot.BaseMap(logo=True,logoPos=[5,88])
map = Plot.MapMesh(cellGrid,map)
map = Plot.MapWaypoints(pd.read_csv(OptInfo['Route']['WayPoints']),map)
map = Plot.LayerControl(map,collapsed=True)
map